# Modeling Workbook

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from math import sqrt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import IsolationForest, RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

In [2]:
from prepare import handle_nulls, set_index
from preprocessing import spotify_split, split_df, scale_data, encode_features

In [3]:
def get_model_features(df):
    '''
    This function takes in a DataFrame and returns a DataFrame with features to use in predictive modeling.
    '''
    df = df.drop(columns=['artist', 'album', 'release_date', 'track_name'])
    return df

---
# Wrangle

In [4]:
df = pd.read_csv('full-playlist.csv', index_col=0)
df = handle_nulls(df)
df = encode_features(df)
df = df.drop(columns='track_id')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6073 entries, 0 to 6073
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            6073 non-null   object 
 1   album             6073 non-null   object 
 2   release_date      6073 non-null   object 
 3   track_name        6073 non-null   object 
 4   danceability      6073 non-null   float64
 5   energy            6073 non-null   float64
 6   key               6073 non-null   float64
 7   loudness          6073 non-null   float64
 8   mode              6073 non-null   float64
 9   speechiness       6073 non-null   float64
 10  instrumentalness  6073 non-null   float64
 11  liveness          6073 non-null   float64
 12  valence           6073 non-null   float64
 13  tempo             6073 non-null   float64
 14  duration_ms       6073 non-null   float64
 15  time_signature    6073 non-null   float64
 16  popularity        6073 non-null   float64


In [6]:
df.head()

,artist,album,release_date,track_name,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,disc_number,is_explicit
0,Tay-K,TRAPMAN,2020-07-12,TRAPMAN,0.792,0.594,2.0,-8.544,1.0,0.3000,0.000000,0.244,0.351,82.512,232803.0,4.0,43.0,1.0,1
1,Lil Wyte,Doubt Me Now,2003-03-04,Oxy Cotton,0.816,0.578,9.0,-6.912,1.0,0.2330,0.000000,0.114,0.265,148.077,193920.0,4.0,61.0,1.0,1
2,Kamelen,KINGPIN SLIM,2019-11-29,Kingpin O.G - Remix,0.649,0.798,0.0,-6.450,0.0,0.1450,0.000000,0.409,0.717,160.011,254390.0,4.0,22.0,1.0,1
3,Waka Flocka Flame,Flockaveli,2010-10-01,Grove St. Party (feat. Kebo Gotti),0.705,0.702,0.0,-4.783,0.0,0.1080,0.000000,0.364,0.771,140.059,250493.0,4.0,62.0,1.0,1
4,Project Pat,Mista Don't Play: Everythangs Workin',2001-02-13,Don't Save Her (feat. Crunchy Black),0.838,0.793,11.0,-5.470,0.0,0.0773,0.000001,0.106,0.800,160.003,261933.0,4.0,45.0,1.0,1


In [7]:
df = get_model_features(df)

In [8]:
# split the data
X_train, y_train, X_validate, y_validate, X_test, y_test, train, validate, test = spotify_split(df, 'popularity')
X_train.head(3)

Shape of train: (4250, 14) | Shape of validate: (912, 14) | Shape of test: (911, 14)
Percent train: 70.0        | Percent validate: 15.0       | Percent test: 15.0


,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,disc_number,is_explicit
3072,0.646,0.595,10.0,-6.709,0.0,0.0512,0.000006,0.0527,0.772,73.973,238880.0,4.0,1.0,0
5674,0.839,0.335,9.0,-14.418,1.0,0.1750,0.000009,0.0967,0.566,127.053,151181.0,4.0,1.0,1
4005,0.517,0.903,10.0,-6.333,0.0,0.5680,0.000000,0.6900,0.643,84.792,196338.0,4.0,1.0,1


### Scale the Data

In [9]:
# MIN-MAX
X_train_mm, X_validate_mm, X_test_mm = scale_data(train, validate, test, 'popularity', 'MinMax')
X_train_mm.head(3)

,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,disc_number,is_explicit
3072,0.655172,0.597154,0.909091,0.727753,0.0,0.053222,0.000006,0.034912,0.786151,0.335858,0.411324,0.8,0.0,0.0
5674,0.850913,0.333988,0.818182,0.411772,1.0,0.181913,0.000010,0.080903,0.576375,0.576855,0.251020,0.8,0.0,1.0
4005,0.524341,0.908904,0.909091,0.743165,0.0,0.590437,0.000000,0.701056,0.654786,0.384979,0.333562,0.8,0.0,1.0


In [10]:
# STANDARD
X_train_st, X_validate_st, X_test_st = scale_data(train, validate, test, 'popularity', 'Standard')
X_train_st.head(3)

,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,disc_number,is_explicit
3072,-0.687128,-0.465816,1.254077,0.084881,-1.156604,-1.257898,-0.155507,-0.938368,1.135696,-1.530581,0.504248,0.007503,-0.066465,-2.109198
5674,0.759798,-2.159084,0.987785,-2.632940,0.864600,-0.403985,-0.155462,-0.667645,0.203152,0.235817,-0.928766,0.007503,-0.066465,0.474114
4005,-1.654244,1.540056,1.254077,0.217441,-1.156604,2.306739,-0.155580,2.982807,0.551724,-1.170546,-0.190894,0.007503,-0.066465,0.474114


In [11]:
# ROBUST
X_train_rb, X_validate_rb, X_test_rb = scale_data(train, validate, test, 'popularity', 'Robust')
X_train_rb.head(3)

,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,disc_number,is_explicit
3072,-0.687128,-0.465816,1.254077,0.084881,-1.156604,-1.257898,-0.155507,-0.938368,1.135696,-1.530581,0.504248,0.007503,-0.066465,-2.109198
5674,0.759798,-2.159084,0.987785,-2.632940,0.864600,-0.403985,-0.155462,-0.667645,0.203152,0.235817,-0.928766,0.007503,-0.066465,0.474114
4005,-1.654244,1.540056,1.254077,0.217441,-1.156604,2.306739,-0.155580,2.982807,0.551724,-1.170546,-0.190894,0.007503,-0.066465,0.474114


--- 
# Feature Selection

In [12]:
# SELECT K BEST
from sklearn.feature_selection import SelectKBest, f_regression

In [14]:
f_selector = SelectKBest(f_regression, 5)
f_selector.fit(X_train_mm, y_train)
f_mask = f_selector.get_support()
X_train_scaled_f_reduced = X_train_mm.iloc[:,f_mask]
X_train_scaled_f_reduced.head(1)

,danceability,energy,loudness,speechiness,is_explicit
3072,0.655172,0.597154,0.727753,0.053222,0.0


In [15]:
skb_features = X_train_scaled_f_reduced

In [16]:
#RFE
from sklearn.feature_selection import RFE

In [17]:
# creat3 the ML model object
lm = LinearRegression()
# create the rfe object
rfe = RFE(lm, 5)
# fit the rfe
rfe.fit(X_train_mm, y_train)
# get the feature support boolean list
rfe_mask = rfe.support_
# reduce the dataframe to just those features
X_train_scaled_rfe_reduced = X_train_mm.iloc[:,rfe_mask]
X_train_scaled_rfe_reduced.head(1)

,danceability,energy,loudness,speechiness,tempo
3072,0.655172,0.597154,0.727753,0.053222,0.335858


In [18]:
rfe_features = X_train_scaled_rfe_reduced

--- 
# Set the baseline

In [19]:
baseline = np.mean(y_train)
baseline

38.46776470588235

In [131]:
baseline_train_rmse = round(sqrt(mean_squared_error(y_train, np.full(len(y_train), baseline))), 6)
print('RMSE (Root Mean Square Error) of Baseline on train data:\n', baseline_train_rmse)

baseline_validate_rmse = round(sqrt(mean_squared_error(y_validate, np.full(len(y_validate), baseline))), 6)
print('RMSE (Root Mean Square Error) of Baseline on validate data:\n', baseline_validate_rmse)

baseline_test_rmse = round(sqrt(mean_squared_error(y_test, np.full(len(y_test), baseline))), 6)
print('RMSE (Root Mean Square Error) of Baseline on test data:\n', baseline_test_rmse)

RMSE (Root Mean Square Error) of Baseline on train data:
 22.770177
RMSE (Root Mean Square Error) of Baseline on validate data:
 23.034868
RMSE (Root Mean Square Error) of Baseline on test data:
 22.875116


---
# MODELS

---
### Cross Validation

In [29]:
from sklearn.model_selection import GridSearchCV

In [114]:
# OLS CV
params = {'fit_intercept': [True, False]}

lm = LinearRegression()

grid = GridSearchCV(lm, params, 
                    scoring= 'neg_root_mean_squared_error', 
                    cv=3, iid=True)

grid.fit(X_train_mm, y_train)

results = grid.cv_results_
#results.keys()

params = results['params']
test_scores = results['mean_test_score']

for p, s in zip(params, test_scores):
    p['RMSE'] = s

pd.DataFrame(params).sort_values(by='RMSE', ascending=False)

,fit_intercept,RMSE
0,True,-21.863341
1,False,-21.872965


In [31]:
# Lasso Lars CV
params = {'fit_intercept': [True, False],
          'alpha': [.0001, .001, .01]
         }
          
lars = LassoLars()

grid = GridSearchCV(lars, params, 
                    scoring= 'neg_root_mean_squared_error', 
                    cv=3, iid=True)

grid.fit(X_train_mm, y_train)

results = grid.cv_results_
#results.keys()

params = results['params']
test_scores = results['mean_test_score']

for p, s in zip(params, test_scores):
    p['RMSE'] = s

pd.DataFrame(params).sort_values(by='RMSE', ascending=False)

,alpha,fit_intercept,RMSE
5,0.0100,False,-21.846209
2,0.0010,True,-21.857063
0,0.0001,True,-21.862590
3,0.0010,False,-21.869652
1,0.0001,False,-21.872626
4,0.0100,True,-21.873841


In [61]:
def PolynomialRegression(degree=2, interaction_only=False, include_bias=True, order='C', **kwargs):
    from sklearn.pipeline import make_pipeline
    return make_pipeline(PolynomialFeatures(degree, interaction_only, include_bias, order),
                         LinearRegression(**kwargs))

In [65]:
PolynomialRegression().get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'polynomialfeatures', 'linearregression', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'polynomialfeatures__order', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__normalize'])

In [75]:
PolynomialRegression()

Pipeline(memory=None,
         steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=2, include_bias=True,
                                    interaction_only=False, order='C')),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [77]:
# Polynomial Features
params = {'polynomialfeatures__degree': [2, 3, 4],
          'polynomialfeatures__interaction_only': [True, False],
          'polynomialfeatures__include_bias': [True, False],
          'polynomialfeatures__order': ['C', 'F']
         }
          
pf = PolynomialRegression()

grid = GridSearchCV(pf, params, 
                    scoring= 'neg_root_mean_squared_error', 
                    cv=3, iid=True)

grid.fit(X_train_mm, y_train)

results = grid.cv_results_
#results.keys()

params = results['params']
test_scores = results['mean_test_score']

for p, s in zip(params, test_scores):
    p['RMSE'] = s

pd.DataFrame(params).sort_values(by='RMSE', ascending=False)

,polynomialfeatures__degree,polynomialfeatures__include_bias,polynomialfeatures__interaction_only,polynomialfeatures__order,RMSE
7,2,False,False,F,-3.151275e+01
6,2,False,False,C,-3.151275e+01
2,2,True,False,C,-3.151307e+01
3,2,True,False,F,-3.153650e+01
1,2,True,True,F,-1.108552e+02
0,2,True,True,C,-1.108552e+02
5,2,False,True,F,-1.108552e+02
4,2,False,True,C,-1.108552e+02
9,3,True,True,F,-1.143331e+02
8,3,True,True,C,-1.143331e+02


---
## Train and Evaluate
### OLS Model
- all features
- min-max scaler

In [22]:
lm = LinearRegression(normalize=True)
lm.fit(X_train_mm, y_train)
lm_pred = lm.predict(X_train_mm)
lm_rmse = sqrt(mean_squared_error(y_train, lm_pred))

lm_pred_v = lm.predict(X_validate_mm)
lm_rmse_v = sqrt(mean_squared_error(y_validate, lm_pred_v))

print('RMSE for OLS using Linear Regression \n')
print('On train data:\n', round(lm_rmse, 6), '\n') 
#print('On validate data:\n', round(lm_rmse_v, 6))

RMSE for OLS using Linear Regression 

On train data:
 21.745633 



### OLS Model
- select k best features (top 5)
- min-max scaler

In [23]:
lm = LinearRegression(normalize=True)
lm.fit(skb_features, y_train)
lm_pred = lm.predict(skb_features)
lm_rmse = sqrt(mean_squared_error(y_train, lm_pred))

lm_pred_v = lm.predict(X_validate_mm[skb_features.columns.to_list()])
lm_rmse_v = sqrt(mean_squared_error(y_validate, lm_pred_v))

print('RMSE for OLS using Linear Regression \n')
print('On train data:\n', round(lm_rmse, 6), '\n') 
#print('On validate data:\n', round(lm_rmse_v, 6))

RMSE for OLS using Linear Regression 

On train data:
 21.806314 



---
### OLS Model
- RFE features (top 5)
- min-max scaler

In [24]:
lm = LinearRegression(normalize=True)
lm.fit(rfe_features, y_train)
lm_pred = lm.predict(rfe_features)
lm_rmse = sqrt(mean_squared_error(y_train, lm_pred))

lm_pred_v = lm.predict(X_validate_mm[rfe_features.columns.to_list()])
lm_rmse_v = sqrt(mean_squared_error(y_validate, lm_pred_v))

print('RMSE for OLS using Linear Regression \n')
print('On train data:\n', round(lm_rmse, 6), '\n') 
#print('On validate data:\n', round(lm_rmse_v, 6))

RMSE for OLS using Linear Regression 

On train data:
 21.913113 



<div class="alert alert-block alert-info">
    <b>Takeaways</b>: using the feature selectors (opposed to using all features) and selecting the top 5 features doesn't improve model performance.</div>

### Lasso + Lars
- all features
- min-max-scaler
- alpha=0.1

In [132]:
lars = LassoLars(alpha=.001)
lars.fit(X_train_mm, y_train)

lars_pred = lars.predict(X_train_mm)
lars_rmse = sqrt(mean_squared_error(y_train, lars_pred))

lars_pred_v = lars.predict(X_validate_mm)
lars_rmse_v = sqrt(mean_squared_error(y_validate, lars_pred_v))

print('RMSE for LASSO + LARS \n')
print('On train data:\n', round(lars_rmse, 6), '\n') 
#print('On validate data:\n', round(lars_rmse_v, 6))

RMSE for LASSO + LARS 

On train data:
 21.746885 



In [133]:
# smaller alpha
lars = LassoLars(alpha=.00001, fit_intercept=True)
lars.fit(X_train_mm, y_train)

lars_pred = lars.predict(X_train_mm)
lars_rmse = sqrt(mean_squared_error(y_train, lars_pred))

lars_pred_v = lars.predict(X_validate_mm)
lars_rmse_v = sqrt(mean_squared_error(y_validate, lars_pred_v))

lars_pred_t = lars.predict(X_test_mm)###
lars_rmse_t = sqrt(mean_squared_error(y_test, lars_pred_t))###

print('RMSE for LASSO + LARS \n')
print('On train data:\n', round(lars_rmse, 6), '\n') 
#print('On validate data:\n', round(lars_rmse_v, 6))

RMSE for LASSO + LARS 

On train data:
 21.745633 



### Polynomial Features + Linear Regression

In [124]:
#squared
pf = PolynomialFeatures(degree=2)
X_train_sq = pf.fit_transform(X_train_mm)
X_validate_sq = pf.transform(X_validate_mm)

lm_sq = LinearRegression()
lm_sq.fit(X_train_sq, y_train)

lm_sq_pred = lm_sq.predict(X_train_sq)
lm_sq_rmse = sqrt(mean_squared_error(y_train, lm_sq_pred))

lm_sq_pred_v = lm_sq.predict(X_validate_sq)
lm_sq_rmse_v = sqrt(mean_squared_error(y_validate, lm_sq_pred_v))

print('RMSE for Polynomial Squared + Linear Regression \n')
print('On train data:\n', round(lm_sq_rmse, 6), '\n') 
#print('On validate data:\n', round(lm_sq_rmse_v, 6))

RMSE for Polynomial Squared + Linear Regression 

On train data:
 21.319629 



In [108]:
# cubed
pf = PolynomialFeatures(degree=3)
X_train_cb = pf.fit_transform(X_train_mm)
X_validate_cb = pf.transform(X_validate_mm)

lm_cb = LinearRegression()
lm_cb.fit(X_train_cb, y_train)

lm_cb_pred = lm_cb.predict(X_train_cb)
lm_cb_rmse = sqrt(mean_squared_error(y_train, lm_cb_pred))

lm_cb_pred_v = lm_cb.predict(X_validate_cb)
lm_cb_rmse_v = sqrt(mean_squared_error(y_validate, lm_cb_pred_v))

print('RMSE for Polynomial Squared + Linear Regression \n')
print('On train data:\n', round(lm_cb_rmse, 6), '\n') 
#print('On validate data:\n', round(lm_cb_rmse_v, 6))

RMSE for Polynomial Squared + Linear Regression 

On train data:
 20.103349 



In [151]:
columns = ['train_rmse', 'validate_rmse', 'test_rmse']
index = ['baseline', 'ols', 'lassolars', 'pf2_lr', 'pf3_lr']
data = [[baseline_train_rmse, baseline_validate_rmse, baseline_test_rmse],
        [lm_rmse, ],
        [lars_rmse, lars_rmse_v, lars_rmse_t],
        [lm_sq_rmse, lm_sq_rmse_v],
        [lm_cb_rmse, lm_cb_rmse_v, ]]
pd.DataFrame(columns=columns, data=data, index=index).sort_values(by='train_rmse')

,train_rmse,validate_rmse,test_rmse
pf3_lr,20.103349,25.876706,NaN
pf2_lr,21.319629,21.708322,NaN
lassolars,21.745633,21.456440,21.924156
ols,21.913113,NaN,NaN
baseline,22.770177,23.034868,22.875116


<div class="alert alert-block alert-info">
    <b>Takeaways</b>:
    <li>Polynomial features cubed is overfit</li>
    <li>Polynomial features squared and Lasso lars perform best on train and validate</li>
    <li><b>Best Model</b> is Lasso Lars since it performed better on unseen data</li>
    <li>Beat the baseline but barely.</li>
    </div>

In [152]:
print(
    f'Model beat baseline by {abs((lars_rmse_t - baseline_test_rmse)/baseline_train_rmse)*100:.2f}%')

Model beat baseline by 4.18%
